In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from src import AllModule, AllDataModule

dm = AllDataModule(batch_size=512, num_workers=10, pin_memory=True)
dm.setup()

train: 1587395
val: 40080
test: 36421


In [3]:
import torch 

state_dict = torch.load('checkpoints/final.ckpt')['state_dict']
model = AllModule(dict(backbone='resnet18', pretrained=True, mlp_layers=[256, 512], mlp_dropout=0.))
model.load_state_dict(state_dict)

<All keys matched successfully>

In [4]:
from tqdm import tqdm
from src.GLC.metrics import top_30_error_rate
import numpy as np 

model.cuda(1)
dl = dm.val_dataloader()
accs = []
for batch in tqdm(dl):
    labels = batch['label']
    preds = model.predict(batch)
    accs.append(top_30_error_rate(labels, preds.cpu()))
np.mean(accs)

100%|██████████| 79/79 [00:57<00:00,  1.37it/s]


0.6210443037974683

In [5]:
from tqdm import tqdm

model.cuda(1)
dl = dm.test_dataloader()
labels, observations = [], []
for batch in tqdm(dl):
    preds = model.predict(batch)
    values, ixs = preds.topk(30)
    labels += [' '.join([str(i.item()) for i in ix]) for ix in ixs]
    observation_ids = batch['observation_id']
    observations += observation_ids.numpy().tolist()

100%|██████████| 72/72 [00:22<00:00,  3.18it/s]


In [6]:
import pandas as pd 

submission = pd.DataFrame({'Id': observations, 'Predicted': labels})
submission.to_csv('submission.csv', index=False)
submission.sample(5)

,Id,Predicted
16350,10758893,727 622 986 645 432 448 195 783 803 459 405 73...
16242,10320893,559 334 600 477 211 146 385 232 1116 660 677 7...
26991,21055492,6038 7146 6245 7144 5754 5613 7748 5775 7575 5...
3624,10681387,600 146 362 359 417 318 599 477 445 677 373 21...
1160,10601370,110 251 949 808 43 473 732 850 798 192 146 521...


In [7]:
sample_submission = pd.read_csv('data/sample_submission.csv')
assert len(sample_submission) == len(submission)
sample_submission.sample(10)

,Id,Predicted
34184,21822962,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
11226,10538678,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2986,10142348,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
15537,10742084,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
5453,10258992,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
23292,20652300,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
32469,21641552,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
17255,10821538,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
18369,20113524,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
19923,20288775,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
